In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import json
import requests
from dotenv import load_dotenv

from td.client import TDClient
import datetime as dt

In [2]:
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
REDIRECT_URI = os.getenv('REDIRECT_URI')
CREDENTIALS_PATH = os.getenv('CREDENTIALS_PATH')

In [3]:
TDSession = TDClient(
    client_id=CLIENT_ID,
    redirect_uri=REDIRECT_URI,
    credentials_path=CREDENTIALS_PATH
)

TDSession.login()

Grabbing new access token...


True

In [4]:
df = pd.DataFrame(TDSession.get_price_history("SPY",start_date=1022648400000)['candles'])

df['datetime'] = df['datetime']/1000
df['datetime'] = df['datetime'].apply(dt.datetime.fromtimestamp)
df = df.set_index('datetime')
df.drop(["open", "high", "low", "volume"], axis=1, inplace=True)
df.dropna(inplace=True)

df.head()

,close
datetime,
2022-04-14 01:12:00,444.56
2022-04-14 01:13:00,444.27
2022-04-14 01:15:00,444.57
2022-04-14 01:16:00,444.39
2022-04-14 01:39:00,444.60


In [5]:
rf = 0.007
path1 = 'chart.csv'
path2 = 'SPY.csv'

In [6]:
def format_ameritrade_export(file_path):
    account = pd.read_csv(
        Path(file_path)
    )
    
    account.rename(columns = {"Account value": "value"}, inplace=True)
    account.dropna(inplace=True)
    account["Date"] = pd.to_datetime(account["Date"], format = '%m/%d/%Y')
    account.set_index("Date", inplace = True)
    #account["value"] = account["value"].str.replace(',', '')
    account["value"] = account["value"].astype(float, errors = 'raise')
    account.dropna(inplace=True)
    
    return account

In [7]:
def format_yfinance_export(file_path):
    spy_data = pd.read_csv(
        Path(file_path),
        index_col = 'Date',
        parse_dates = True,
        infer_datetime_format = True
    )
    
    spy_data.drop(["Open", "High", "Low", "Adj Close", "Volume"], axis=1, inplace=True)
    spy_data.dropna(inplace=True)
    
    return spy_data
    

In [87]:
def get_statistics(portfolio_data, spy_data, rf):
    statistics = ["daily_returns", "cumulative_returns", "average_annualized_returns", "volatility", "variance", "covariance", "annualized_sharpe_ratio"] 
    portfolio_statistics = [] 
    market_statistics = []
    rows = ["portfolio", "market"]
    
    daily_returns = portfolio_data["value"].pct_change()
    spy_daily_returns = spy_data["Close"].pct_change()
    portfolio_statistics.append(daily_returns)
    market_statistics.append(spy_daily_returns)
    
    cum_returns = (1+daily_returns).cumprod()
    spy_cum_returns = (1+spy_daily_returns).cumprod()
    portfolio_statistics.append(cum_returns)
    market_statistics.append(spy_cum_returns)
    
    average_annualized_returns = daily_returns.mean() * 252
    average_annualized_market_returns = spy_daily_returns.mean() * 252
    portfolio_statistics.append(average_annualized_returns)
    market_statistics.append(average_annualized_market_returns)
    
    volatility = portfolio_data["value"].std()
    market_volatility = spy_daily_returns.std()
    portfolio_statistics.append(volatility)
    market_statistics.append(market_volatility)
    
    variance = daily_returns.var()
    market_variance = spy_daily_returns.var()
    portfolio_statistics.append(variance)
    market_statistics.append(market_variance)
    
    covariance = daily_returns.cov(spy_daily_returns)
    portfolio_statistics.append(covariance)
    market_statistics.append(covariance)
    
    annualized_volatility = volatility * np.sqrt(252)
    annualized_market_volatility = market_volatility * np.sqrt(252)
    sharpe_ratio = average_annualized_returns - rf / annualized_volatility
    market_sharpe_ratio = average_annualized_market_returns - rf / annualized_market_volatility
    portfolio_statistics.append(sharpe_ratio)
    market_statistics.append(market_sharpe_ratio)
    
    main = [statistics, portfolio_statistics, market_statistics]
    
    summary = pd.DataFrame({'statistics': statistics[2:7], 'portfolio_statistics' : portfolio_statistics[2:7], 'market_statistics' : market_statistics[2:7]}).set_index('statistics')

    return summary

In [88]:
portfolio = format_ameritrade_export(path1)
market = format_yfinance_export(path2)
go = get_statistics(portfolio, market, rf)

In [89]:
go

,portfolio_statistics,market_statistics
statistics,,
average_annualized_returns,0.177592,0.176081
volatility,2162.339266,0.011305
variance,0.000089,0.000128
covariance,0.000053,0.000053
annualized_sharpe_ratio,0.177592,0.137076


In [86]:
new_df = pd.DataFrame({'statistics': go[0][2:7], 'portfolio_statistics' : go[1][2:7], 'market_statistics' : go[2][2:7]}).set_index('statistics')
new_df

,portfolio_statistics,market_statistics
statistics,,
average_annualized_returns,0.177592,0.176081
volatility,2162.339266,0.011305
variance,0.000089,0.000128
covariance,0.000053,0.000053
annualized_sharpe_ratio,0.177592,0.137076


In [27]:
endpoint = 'https://api.tdameritrade.com/v1/marketdata/{stock_ticker}/pricehistory?periodType={periodType}&period={period}&frequencyType={frequencyType}'
full_url = endpoint.format(stock_ticker='SPY',periodType='year',period=20,frequencyType='daily')
page = requests.get(url=full_url,
                    params={'apikey' : os.getenv('CLIENT_ID')})
content = json.loads(page.content)
df = pd.DataFrame(content['candles'])
df['datetime'] = df['datetime']/1000
df['datetime'] = df['datetime'].apply(dt.datetime.fromtimestamp)
df['datetime'] = df['datetime'].dt.date
df = df.set_index('datetime')
df.drop(["open", "high", "low", "volume"], axis=1, inplace=True)
df.dropna(inplace=True)
df

,close
datetime,
2002-05-28,108.10
2002-05-29,107.30
2002-05-30,107.00
2002-05-31,107.22
2002-06-03,104.37
...,...
2022-05-23,396.92
2022-05-24,393.89
2022-05-25,397.37


In [20]:
content

{'candles': [{'open': 109.05,
   'high': 109.13,
   'low': 107.45,
   'close': 108.1,
   'volume': 24236900,
   'datetime': 1022562000000},
  {'open': 107.62,
   'high': 108.02,
   'low': 107.13,
   'close': 107.3,
   'volume': 14773300,
   'datetime': 1022648400000},
  {'open': 106.55,
   'high': 107.51,
   'low': 105.9,
   'close': 107.0,
   'volume': 18217900,
   'datetime': 1022734800000},
  {'open': 107.4,
   'high': 108.56,
   'low': 106.85,
   'close': 107.22,
   'volume': 19826300,
   'datetime': 1022821200000},
  {'open': 107.09,
   'high': 107.6,
   'low': 104.13,
   'close': 104.37,
   'volume': 26056300,
   'datetime': 1023080400000},
  {'open': 104.15,
   'high': 105.2,
   'low': 103.55,
   'close': 104.63,
   'volume': 25856200,
   'datetime': 1023166800000},
  {'open': 104.95,
   'high': 105.67,
   'low': 104.35,
   'close': 105.61,
   'volume': 19695900,
   'datetime': 1023253200000},
  {'open': 105.54,
   'high': 105.6,
   'low': 103.15,
   'close': 103.46,
   'volume'

In [ ]:
endpoint = f'https://api.tdameritrade.com/v1/marketdata/{stock_ticker}/quotes?'
full_url = endpoint.format(stock_ticker='AAL')
page = requests.get(url=full_url,
                    params={'apikey' : td_consumer_key})
content = json.loads(page.content)